In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_16_1"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [7]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 2, 2]
weights_shape = [[9, n_variables, 128],
                 [5, 128, 128],
                 [5, 128, 128]]
bias_shape = [128, 128, 128]
wavelet_weights_shape = [[3, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [9]:
train_data.shape

(691, 1024, 77)

In [10]:
pos_weight.shape

(19,)

In [11]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet = True,
                                 trainable_conv = True,
                                 trainable_last = True,
                                 variables_weights = variables_weights)

In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [17]:
trainer.train(max_iter = 3000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.062
F1 = 0.100 vs. 0.094
Accuracy = 0.000 vs. 0.056
CHANGE DETECTION
AUC = 0.104 vs. 0.103
F1 = 0.221 vs. 0.181
Precision = 0.126 vs. 0.100
Recall = 0.888 vs. 0.997
Validation: 0
ACTIVITY RECOGNITION
AUC = 0.058 vs. 0.060
F1 = 0.083 vs. 0.085
Accuracy = 0.000 vs. 0.056
CHANGE DETECTION
AUC = 0.062 vs. 0.051
F1 = 0.144 vs. 0.097
Precision = 0.079 vs. 0.051
Recall = 0.787 vs. 0.953
Minibatch Loss= 1.446536
Training: 100
ACTIVITY RECOGNITION
AUC = 0.599 vs. 0.061
F1 = 0.627 vs. 0.088
Accuracy = 0.764 vs. 0.056
CHANGE DETECTION
AUC = 0.265 vs. 0.079
F1 = 0.342 vs. 0.145
Precision = 0.228 vs. 0.078
Recall = 0.684 vs. 0.982
Validation: 100
ACTIVITY RECOGNITION
AUC = 0.430 vs. 0.058
F1 = 0.481 vs. 0.079
Accuracy = 0.727 vs. 0.056
CHANGE DETECTION
AUC = 0.209 vs. 0.054
F1 = 0.297 vs. 0.103
Precision = 0.222 vs. 0.059
Recall = 0.449 vs. 0.410
Minibatch Loss= 0.297502
Training: 200
ACTIVITY RECOGNITION
AUC = 0.814 vs. 0.060
F1 = 0.780 vs. 0.089


In [18]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_16_1/checkpoint-3000'

In [19]:
trainer.train(max_iter = 3000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 3000
ACTIVITY RECOGNITION
AUC = 0.999 vs. 0.062
F1 = 0.997 vs. 0.092
Accuracy = 0.995 vs. 0.056
CHANGE DETECTION
AUC = 0.956 vs. 0.089
F1 = 0.958 vs. 0.170
Precision = 0.923 vs. 0.093
Recall = 0.995 vs. 1.000
Validation: 3000
ACTIVITY RECOGNITION
AUC = 0.642 vs. 0.060
F1 = 0.640 vs. 0.085
Accuracy = 0.911 vs. 0.056
CHANGE DETECTION
AUC = 0.296 vs. 0.049
F1 = 0.360 vs. 0.097
Precision = 0.274 vs. 0.051
Recall = 0.527 vs. 0.977
Minibatch Loss= 0.008282
Training: 3100
ACTIVITY RECOGNITION
AUC = 1.000 vs. 0.060
F1 = 0.999 vs. 0.088
Accuracy = 0.995 vs. 0.056
CHANGE DETECTION
AUC = 0.951 vs. 0.078
F1 = 0.940 vs. 0.147
Precision = 0.916 vs. 0.081
Recall = 0.967 vs. 0.824
Validation: 3100
ACTIVITY RECOGNITION
AUC = 0.638 vs. 0.058
F1 = 0.639 vs. 0.078
Accuracy = 0.912 vs. 0.056
CHANGE DETECTION
AUC = 0.298 vs. 0.052
F1 = 0.369 vs. 0.099
Precision = 0.284 vs. 0.055
Recall = 0.527 vs. 0.538
Minibatch Loss= 0.007521
Training: 3200
ACTIVITY RECOGNITION
AUC = 1.000 vs. 0.062
F1 = 0.999 v

In [20]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_16_1/checkpoint-6000'

In [21]:
trainer.train(max_iter = 3000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 6000
ACTIVITY RECOGNITION
AUC = 1.000 vs. 0.062
F1 = 1.000 vs. 0.092
Accuracy = 1.000 vs. 0.056
CHANGE DETECTION
AUC = 0.999 vs. 0.099
F1 = 0.998 vs. 0.174
Precision = 0.997 vs. 0.097
Recall = 1.000 vs. 0.888
Validation: 6000
ACTIVITY RECOGNITION
AUC = 0.673 vs. 0.059
F1 = 0.665 vs. 0.082
Accuracy = 0.937 vs. 0.056
CHANGE DETECTION
AUC = 0.312 vs. 0.051
F1 = 0.403 vs. 0.098
Precision = 0.316 vs. 0.052
Recall = 0.553 vs. 0.860
Minibatch Loss= 0.001026
Training: 6100
ACTIVITY RECOGNITION
AUC = 1.000 vs. 0.062
F1 = 1.000 vs. 0.093
Accuracy = 1.000 vs. 0.056
CHANGE DETECTION
AUC = 1.000 vs. 0.095
F1 = 1.000 vs. 0.180
Precision = 1.000 vs. 0.099
Recall = 1.000 vs. 0.952
Validation: 6100
ACTIVITY RECOGNITION
AUC = 0.673 vs. 0.059
F1 = 0.663 vs. 0.080
Accuracy = 0.938 vs. 0.056
CHANGE DETECTION
AUC = 0.313 vs. 0.052
F1 = 0.397 vs. 0.100
Precision = 0.328 vs. 0.053
Recall = 0.504 vs. 0.797
Minibatch Loss= 0.001048
Training: 6200
ACTIVITY RECOGNITION
AUC = 0.904 vs. 0.061
F1 = 0.849 v

In [22]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../../model/DWN_16_1/checkpoint-9000'

In [19]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../../model/DWN_Wmultitask_16_opp/checkpoint-20000')


In [26]:
#load the model
saver = tf.train.Saver()
#'../../model/DWN_16_1/checkpoint-6000'
saver.restore(sess, "../../model/DWN_16_1/checkpoint-6000")

INFO:tensorflow:Restoring parameters from ../../model/DWN_16_1/checkpoint-6000


In [27]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.673 vs. 0.058
F1 = 0.665 vs. 0.077
Accuracy = 0.937 vs. 0.056
CHANGE DETECTION
AUC = 0.312 vs. 0.055
F1 = 0.403 vs. 0.101
Precision = 0.316 vs. 0.056
Recall = 0.553 vs. 0.561


In [28]:
out = nn_eval.deploy(model, sess, test_data, test_gt)

In [29]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(out, axis = 2)

In [30]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})